[View in Colaboratory](https://colab.research.google.com/github/clear21/Word-calculator/blob/add-new-file/word2vec_text8.ipynb)

In [1]:
#サンプルコード
#https://github.com/oreilly-japan/deep-learning-with-keras-ja

#MeCabのインストール準備
#https://qiita.com/pytry3g/items/897ae738b8fbd3ae7893
# !apt install aptitude
# !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
# !pip install mecab-python3

#genismのインストール
!pip install gensim

import urllib.request
from gensim.models import word2vec 

!pip install janome

#英語の形態素解析
!pip install nltk

#google翻訳
!pip install googletrans

    100% |████████████████████████████████| 23.6MB 1.4MB/s 
    100% |████████████████████████████████| 1.4MB 13.5MB/s 
    100% |████████████████████████████████| 133kB 18.7MB/s 
    100% |████████████████████████████████| 4.7MB 5.4MB/s 
    100% |████████████████████████████████| 61kB 20.1MB/s 
    100% |████████████████████████████████| 552kB 21.6MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
    100% |████████████████████████████████| 20.0MB 1.8MB/s 
  Running setup.py bdist_wheel for janome ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/

In [2]:
#MeCabのインストール準備
#https://qiita.com/pytry3g/items/897ae738b8fbd3ae7893
# !apt install aptitude
# !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
# !pip install mecab-python3

import urllib.request

#モデル作成は使わず、text8を単語毎に分割するために使用
from gensim.models import word2vec 

from keras.models import Sequential
from keras.layers import Merge , Dense , Reshape , Embedding

#Tokenizer使用のため
from keras.preprocessing.text import *
from keras.preprocessing.sequence import skipgrams
from keras.utils import to_categorical
import keras.backend as K

import numpy as np
from numpy.linalg import norm

import pandas as pd

#日本語形態素解析
#import janome.tokenizer

#英語形態素解析（natural langage tool kit）
import nltk
nltk.download('averaged_perceptron_tagger')

#google翻訳
from googletrans import Translator

import json

from google.colab import files

import pickle

Using TensorFlow backend.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
!python -c "help('janome')"

Help on package janome:

NNAAMMEE
    janome

PPAACCKKAAGGEE  CCOONNTTEENNTTSS
    analyzer
    charfilter
    dic
    fst
    lattice
    tokenfilter
    tokenizer

FFIILLEE
    /usr/local/lib/python3.6/dist-packages/janome/__init__.py




In [4]:
#text8コーパスのダウンロード
url = r'http://mattmahoney.net/dc/text8.zip'
save_file_name = 'text8.zip'

urllib.request.urlretrieve(url , save_file_name)
print('success')

#解凍
!ls
!unzip -o text8.zip
!rm text8.zip
!ls

success
sample_data  text8.zip
Archive:  text8.zip
  inflating: text8                   
sample_data  text8


In [5]:
#50語（第二引数）毎の文に分ける
sentences = word2vec.Text8Corpus("text8")
sentences_list_tmp = list(sentences)
print(len(sentences_list_tmp))

1701


In [6]:
#メモリ負荷削減
print('len(sentences_list_tmp)' , len(sentences_list_tmp))
sentences_list_tmp = sentences_list_tmp[:300]
print('len(sentences_list_tmp)' , len(sentences_list_tmp))

len(sentences_list_tmp) 1701
len(sentences_list_tmp) 300


In [7]:
#品詞確認用
#https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
#【説明】http://computer-technology.hateblo.jp/entry/20150824/p1
pos_df = pd.DataFrame(
[
[1,'CC','Coordinating conjunction'] #等位接続詞
,[2,'CD','Cardinal number'] #基数
,[3,'DT','Determiner'] 
,[4,'EX','Existential there ']
,[5,'FW','Foreign word']
,[6,'IN','Preposition or subordinating conjunction']
,[7,'JJ','Adjective']
,[8,'JJR','Adjective, comparative']
,[9,'JJS','Adjective, superlative']
,[10,'LS','List item marker']
,[11,'MD','Modal']
,[12,'NN','Noun, singular or mass']
,[13,'NNS','Noun, plural']
,[14,'NNP','Proper noun, singular']
,[15,'NNPS','Proper noun, plural']
,[16,'PDT','Predeterminer']
,[17,'POS','Possessive ending']
,[18,'PRP','Personal pronoun']
,[19,'PRP$','Possessive pronoun']
,[20,'RB','Adverb']
,[21,'RBR','Adverb, comparative']
,[22,'RBS','Adverb, superlative']
,[23,'RP','Particle']
,[24,'SYM','Symbol']
,[25,'TO','to']
,[26,'UH','Interjection']
,[27,'VB','Verb, base form']
,[28,'VBD','Verb, past tense']
,[29,'VBG','Verb, gerund or present participle']
,[30,'VBN','Verb, past participle']
,[31,'VBP','Verb, non-3rd person singular present']
,[32,'VBZ','Verb, 3rd person singular present']
,[33,'WDT','Wh-determiner']
,[34,'WP','Wh-pronoun']
,[35,'WP$','Possessive wh-pronoun']
,[36,'WRB','Wh-adverb']

]
, columns = ['Number' , 'Tag' , 'Description'])


#抽出する品詞
p_o_s_list = [
# 'FW' #外国語
# ,'JJ' #形容詞
# ,'JJR' #形容詞の比較級
# ,'JJS' #形容詞の最上級
    
'NN' #名詞（単数形または不可算名詞）
,'NNS' #名詞（複数形）
,'NP' #固有名詞（単数形）
,'NPS' #固有名詞（複数形）

    # ,'RB' #副詞
# ,'RBR' #副詞の比較級
# ,'RBS' #副詞の最上級
]


#文に対して、対象の品詞のみを出力
#引数：1文（単語分解済）、抽出する品詞のリスト（pos_tagの品詞）
#戻り値：第一引数から対象の品詞のみを抽出したリスト
def extract_part_of_speech(sentence , p_o_s_list):
    #形態素解析
    morphological_analysis = nltk.pos_tag(sentence)
    #対象の品詞のみ抽出
    sentence_only_given_pos_tmp = list(filter(lambda x : x[1] in p_o_s_list , morphological_analysis))
    #単語のリストにする
    sentence_only_given_pos = [x[0] for x in sentence_only_given_pos_tmp]
    
    return sentence_only_given_pos

#単語、単語IDの辞書

#kerasのTokenizerは品詞解析できなさそう。
tokenizer = Tokenizer()

#janomeのTokenizer
#tokenizer = janome.tokenizer.Tokenizer()

#対象品詞のみ抽出
sentences_list = list(map(lambda sentence : extract_part_of_speech(sentence , p_o_s_list) , sentences_list_tmp))

#Word2ID、ID2Word辞書作成
tokenizer.fit_on_texts(sentences_list)

word2id = tokenizer.word_index
id2word = {key : word for word , key in word2id.items()}

#文の各単語をidに変換したもの
wids = [word2id[w] for sentence in sentences_list for w in sentence]

#pairs：文脈語と中心語のリスト　、　labels：pairが存在する⇒1、存在しない⇒0
pairs , labels = skipgrams(wids , len(word2id) , window_size = 1)

print('len(labels)' , len(labels))

len(labels) 3648184


In [8]:
print(len(list(filter(lambda x : x == 1 , labels))))
print(len(list(filter(lambda x : x == 0 , labels))))

1824092
1824092


In [0]:
del sentences , sentences_list , sentences_list_tmp

In [0]:
!rm text8

In [0]:
#◆jsonファイルで保存
#引数：保存対象オブジェクト、ファイル名(拡張子なし)
def save_as_json(obj , fname):
    json_fname_with_extension = fname + '.json'
    fw = open(json_fname_with_extension , 'w')
    json.dump(obj , fw , indent = 4)
    fw.close()
    files.download(json_fname_with_extension)

#◆jsonファイルを取得
#引数：対象ファイルのパス
def import_json(fpass):
    fr = open(fpass , 'r')
    f = json.load(fr)
    fr.close()
    return f

#◆jsonファイルで保存
#save_as_json(word2id , 'word2id')
#save_as_json(id2word , 'id2word')
#save_as_json(wids , 'wids')
#save_as_json(pairs , 'pairs')
#save_as_json(labels , 'labels')


#◆jsonファイルの読み込み
#files.upload()
#word2id = import_json('word2id.json')
#wids = import_json('wids.json')
#id2word = import_json('id2word.json')

In [0]:
#!rm word2id.json wids.json id2word.json

In [13]:
#単語IDが０でなく、１始まりなのでnum_classesは「+1」する
vocab_size = len(word2id) + 1
embed_size = 200
BATCH_SIZE = vocab_size

#中心語用
word_model = Sequential()
word_model.add(Embedding(vocab_size , embed_size 
                          ,embeddings_initializer = "glorot_uniform"
                          ,input_length = 1))
#教科書だと下記も記載するが、fitで以下のエラーが出る
#ValueError: Error when checking target: expected dense_9 to have 3 dimensions, but got array with shape (19996, 1)
#word_model.add(Reshape((embed_size , 1)))

print(word_model.summary())
print('*' * 30)

#文脈語用
context_model = Sequential()
context_model.add(Embedding(vocab_size , embed_size
                              ,embeddings_initializer = "glorot_uniform"
                              ,input_length = 1))
#教科書だと下記も記載するが、fitで以下のエラーが出る
#ValueError: Error when checking target: expected dense_9 to have 3 dimensions, but got array with shape (19996, 1)
#context_model.add(Reshape((embed_size , 1)))

print(context_model.summary())
print('*' * 30)

#まとめ用
model = Sequential()
model.add(Merge([word_model , context_model] , mode = "dot"))
model.add(Dense(1 , kernel_initializer = "glorot_uniform"
                  , activation = "sigmoid"))

print(model.summary())

model.compile(loss = "mean_squared_error" , optimizer = "adam")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 200)            13851800  
Total params: 13,851,800
Trainable params: 13,851,800
Non-trainable params: 0
_________________________________________________________________
None
******************************
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1, 200)            13851800  
Total params: 13,851,800
Trainable params: 13,851,800
Non-trainable params: 0
_________________________________________________________________
None
******************************
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 1, 1)              0         
______________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [14]:
#単語IDが０でなく、１始まりなのでnum_classesは「+1」する
# words = np.array([to_categorical(x[0] , num_classes = len(word2id) + 1) for x in pairs])
# contexts = np.array([to_categorical(x[1] , num_classes = len(word2id) + 1) for x in pairs])
# pairs = pairs[:5000000]
# labels = labels[:5000000]
words = np.array([x[0] for x in pairs])
contexts = np.array([x[1] for x in pairs])

model.fit([words , contexts] , np.array([[[x]] for x in labels]) , epochs = 1 , verbose = 1)

Epoch 1/1
3648184/3648184 [==============================] - 4187s 1ms/step - loss: 0.1244


In [0]:
#◆結果が返ってこない
# model_fpath = r'word2vec_text8_20180926_sentence5000.hdf5'
# model.save(model_fpath)
# files.download(model_fpath)

In [0]:
#重み取得
merge_layer = model.layers[0]
word_model = merge_layer.layers[0]
word_embed_layer = word_model.layers[0]
weights = word_embed_layer.get_weights()[0]

In [39]:
#◆結果が返ってこない「Service Worker Response Error」
# word_model_fpath = r'word_model_20180926_sentence5000.hdf5'
# word_model.save(word_model_fpath)
# files.download(word_model_fpath)

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 59254, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

MessageError: ignored

In [0]:
#pickleでファイル保存
#引数：保存対象オブジェクト、ファイル名(拡張子なし)
def save_as_pickle(obj , fname):
    fname_with_extension = fname + '.txt'
    fw = open(fname_with_extension , 'wb')
    pickle.dump(obj , fw)
    fw.close()
    files.download(fname_with_extension)

#pickleデータを取得
#引数：対象ファイルのパス
def import_pickle(fpass):
    fr = open(fpass , 'rb')
    f = pickle.load(fr)
    fr.close()
    return f
    
#save_as_pickle(weights , 'weights')

In [0]:
#save_as_json(word2id , 'word2id')
#save_as_json(id2word , 'id2word')

In [0]:
#各単語の分散表現取得
#引数：単語toIDの辞書、重み行列
#戻り値；単語to分散表現のリスト
def get_distributed_representation(word2id , weights):
    word2disvec = {}
    for word , wid in word2id.items():
        #one-hot化
        oh_vec = to_categorical(wid , num_classes = len(word2id) + 1)
        #分散表現
        word2disvec[word] = np.dot(oh_vec , weights)
    
    return word2disvec
    
    
#関連単語top（単語が引数）
#引数：対象単語、単語to分散表現のリスト（get_distributed_representationの戻り値）、出力する関連単語上位 n 
def related_word_top(given_word , word2disvec , top_num):
    #対象単語の分散表現取得
    given_disvec = word2disvec[given_word]
    
    #キー：単語、値：コサイン類似度　の辞書。（対象単語以外）
    dict_cos_degree = {}
    for word , disvec in word2disvec.items():
        if word == given_word:
            continue
        else:
            cos_degree = np.dot(given_disvec , disvec) / (norm(given_disvec) * norm(disvec))
            dict_cos_degree[word] = cos_degree
            
            #print('word' , word , 'cos_degree' ,  cos_degree)
    
    #リスト型
    return sorted(dict_cos_degree.items() , key = lambda x : -x[1])[:top_num]
            
#関連単語top（ベクトルが引数）
#引数：対象ベクトル、単語to分散表現のリスト（get_distributed_representationの戻り値）、出力する関連単語上位 n 
def related_word_top_for_given_disvec(given_disvec , word2disvec , top_num):
    #対象単語の分散表現取得
    #given_disvec = word2disvec[given_word]
    
    #キー：単語、値：コサイン類似度　の辞書。（対象単語以外）
    dict_cos_degree = {}
    for word , disvec in word2disvec.items():
        if disvec is given_disvec:
            continue
        else:
            cos_degree = np.dot(given_disvec , disvec) / (norm(given_disvec) * norm(disvec))
            dict_cos_degree[word] = cos_degree
            
            #print('word' , word , 'cos_degree' ,  cos_degree)
    
    #リスト型
    return sorted(dict_cos_degree.items() , key = lambda x : -x[1])[:top_num]     
    

In [0]:
word2disvec = get_distributed_representation(word2id , weights)

In [0]:
save_as_pickle(word2disvec , 'word2disvec')

In [6]:
#[1_1] ローカルからデータup → 単語2分散表現の作成 → ローカルにDL

#ローカルからファイルup
# print('word2id の jsonファイル を指定してください')
# files.upload()

word2id の jsonファイル を指定してください


KeyboardInterrupt: ignored

In [0]:
#[1_2] ローカルからデータup → 単語2分散表現の作成 → ローカルにDL
# print('weights(重み行列) の txtファイル を指定してください')
# files.upload()

In [7]:
#[2] ローカルからデータup → 単語2分散表現の作成 → ローカルにDL

#upしたファイルをオブジェクト化
#word2id_local = import_json(r'word2id.json')
#weights_local = import_pickle(r'weights.txt')

#単語2分散表現の作成
#word2disvec = get_distributed_representation(word2id_local , weights_local)

#ローカルにDL
# files.download(word2disvec)

UnicodeDecodeError: ignored

In [48]:
a = get_distributed_representation(word2id , weights)

b = related_word_top('process' , a , 10)
for x in b:
    print(x[0] , x[1])
    
print('*' * 30)

b = related_word_top_for_given_disvec(a['process'] , a , 10)
for x in b:
    print(x[0] , x[1])

neurodevelopmental 0.48193133
profitable 0.4788513
turkestan 0.45455354
dialectic 0.44267493
deny 0.44231147
authority 0.4407144
caliphate 0.42183903
corridors 0.41965437
groupoid 0.41963297
complaint 0.41603127
******************************
OK
neurodevelopmental 0.48193133
profitable 0.4788513
turkestan 0.45455354
dialectic 0.44267493
deny 0.44231147
authority 0.4407144
caliphate 0.42183903
corridors 0.41965437
groupoid 0.41963297
complaint 0.41603127


In [40]:
#word2id
print(len(word2id))
print(word2id)

17821
{'zero': 1, 's': 2, 'other': 3, 'many': 4, 'such': 5, 'states': 6, 'world': 7, 'time': 8, 'first': 9, 'people': 10, 'state': 11, 'new': 12, 'lincoln': 13, 'united': 14, 'war': 15, 'american': 16, 'years': 17, 'government': 18, 'history': 19, 'gore': 20, 'north': 21, 'africa': 22, 'human': 23, 'century': 24, 'more': 25, 'part': 26, 'national': 27, 'south': 28, 'einstein': 29, 'alaska': 30, 'most': 31, 'th': 32, 'number': 33, 'i': 34, 'name': 35, 'system': 36, 'apollo': 37, 'autism': 38, 'population': 39, 'president': 40, 'language': 41, 'life': 42, 'year': 43, 'general': 44, 'country': 45, 'major': 46, 'anchorage': 47, 'philosophy': 48, 'several': 49, 'groups': 50, 'different': 51, 'east': 52, 'second': 53, 'work': 54, 'high': 55, 'example': 56, 'm': 57, 'australia': 58, 'order': 59, 'early': 60, 'social': 61, 'science': 62, 'languages': 63, 'area': 64, 'angola': 65, 'use': 66, 'c': 67, 'asia': 68, 'political': 69, 'university': 70, 'u': 71, 'modern': 72, 'theory': 73, 'same': 74,

In [0]:
word2disvec = get_distributed_representation(word2id , weights)

given_disvec = word2disvec[given_word]
related_word_top_for_given_disvec(given_disvec , weights , word2id)

sample_data


In [0]:
####################################################################################################

In [0]:
#import MeCab

今日 は いい 天気 です ね 



    100% |████████████████████████████████| 23.5MB 677kB/s 
    100% |████████████████████████████████| 1.4MB 13.8MB/s 
    100% |████████████████████████████████| 133kB 26.2MB/s 
    100% |████████████████████████████████| 61kB 21.3MB/s 
    100% |████████████████████████████████| 4.7MB 915kB/s 
    100% |████████████████████████████████| 552kB 24.0MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /root/.cache/pip/wheels/73/f1/9b/ccf93d4ba073b6f79b1ed9df68ab5ce048d8136d0efcf90b30
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [0]:
http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Books_5.json.gz

In [0]:
import urllib.request
import json

In [4]:
#url = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Books_5.json.gz"
url = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Movies_and_TV_5.json.gz"
savename = "reviews_Movies_and_TV_5.json.gz"

#ダウンロード --- （＊3）
urllib.request.urlretrieve(url, savename)
print("保存しました")

保存しました


In [5]:
!ls
#!gzip -d reviews_Books_5.json.gz
!gzip -d reviews_Movies_and_TV_5.json.gz

reviews_Movies_and_TV_5.json.gz  sample_data


In [6]:
!ls

reviews_Movies_and_TV_5.json  sample_data


In [0]:
a = open(r'reviews_Movies_and_TV_5.json' , 'r')

temp = {i : json.loads(line) for i , line in list(enumerate(a))}

In [0]:
!rm reviews_Books_5.json

In [19]:
temp[1]

{'asin': '0005019281',
 'helpful': [0, 0],
 'overall': 3.0,
 'reviewText': 'It was good but not as emotionally moving as the the Christmas Carol by Dickens  I like Christmas movies that make me sigh',
 'reviewTime': '12 30, 2013',
 'reviewerID': 'A35947ZP82G7JH',
 'reviewerName': 'Amarah Strack',
 'summary': 'Good but not as moving',
 'unixReviewTime': 1388361600}

In [20]:
url = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Movies_and_TV.json.gz"
savename = "meta_Movies_and_TV.json.gz"

#ダウンロード --- （＊3）
urllib.request.urlretrieve(url, savename)
print("保存しました")

保存しました


In [21]:
!ls
!gzip -d meta_Movies_and_TV.json.gz

meta_Movies_and_TV.json.gz  reviews_Movies_and_TV_5.json  sample_data


In [0]:
b = open(r'meta_Movies_and_TV.json')
temp_2 = {i : json.loads(json.dumps(line)) for i , line in enumerate(b)}

In [58]:
temp_2

{0: "{'asin': '0000143561', 'categories': [['Movies & TV', 'Movies']], 'description': '3Pack DVD set - Italian Classics, Parties and Holidays.', 'title': 'Everyday Italian (with Giada de Laurentiis), Volume 1 (3 Pack): Italian Classics, Parties, Holidays', 'price': 12.99, 'salesRank': {'Movies & TV': 376041}, 'imUrl': 'http://g-ecx.images-amazon.com/images/G/01/x-site/icons/no-img-sm._CB192198896_.gif', 'related': {'also_viewed': ['B0036FO6SI', 'B000KL8ODE', '000014357X', 'B0037718RC', 'B002I5GNVU', 'B000RBU4BM'], 'buy_after_viewing': ['B0036FO6SI', 'B000KL8ODE', '000014357X', 'B0037718RC']}}\n",
 1: '{\'asin\': \'0000589012\', \'title\': "Why Don\'t They Just Quit? DVD Roundtable Discussion: What Families and Friends need to Know About Addiction and Recovery", \'price\': 15.95, \'imUrl\': \'http://ecx.images-amazon.com/images/I/519%2B1kseM3L._SY300_.jpg\', \'related\': {\'also_bought\': [\'B000Z3N1HQ\', \'0578045427\', \'B007VI5AQ8\', \'B003AC98V2\', \'B004V4RW8O\', \'B000I0QL7I\', \'